*YOLO is a deep learning algorythm. So it doesn’t need any installation, instead it needs a deep learning freamwork to run on!*

*Three most know used freamwork compatible with YOLO are -* <br/>
*1. Darknet* <br/>
*2. Darkflow* <br/>
*3. Opencv*

*Imports necessary packages*

In [1]:
import numpy as np
import cv2
import os

*Load the COCO class labels our YOLO model was trained on*

In [2]:
lblPath = "./yolo-coco/coco.names"
LABELS = open(lblPath).read().strip().split("\n")

for i in range(3):
    print(LABELS[i])

person
bicycle
car


*Initialize a list of colors to represent each possible class label*

In [3]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

*Load our YOLO object detector model trained on COCO dataset (80 classes). To run the algorythm we need 2 files:*

*1. Weight file: it’s the trained model, the core of the algorythm to detect the objects.* <br/>
*2. Cfg file: it’s the configuration file, where there are all the settings of the algorythm.* <br/>

*Determine only the output layer names that we need from YOLO*

In [4]:
weightsPath = "./yolo-coco/yolov3.weights"
configPath = "./yolo-coco/yolov3.cfg"

net = cv2.dnn.readNetFromDarknet(configPath,weightsPath)

out_layers = net.getLayerNames()
out_layers = [out_layers[i[0] - 1] for i in net.getUnconnectedOutLayers()]

*Grab video stream*

In [5]:
#vs = cv2.VideoCapture(0)
vs = cv2.VideoCapture("./videos/demo_video.mp4")
(W, H) = (None, None)

In [6]:
frame_width = int(vs.get(3)) 
frame_height = int(vs.get(4))
   
size = (frame_width, frame_height)
result = cv2.VideoWriter("./output/out_video.avi", cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

In [7]:
while True:
    
    _,frame = vs.read()
    
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(out_layers)

    boxes = []
    confidences = []
    classIDs = []
    
    for output in layerOutputs:
        for detection in output:

            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > 0.5:
                (centerX, centerY, width, height) = box = detection[0:4] * np.array([W, H, W, H]).astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # ensure at least one detection exists
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = str(LABELS[classIDs[i]]) + " : " + str(confidences[i])
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    result.write(frame) 
    cv2.imshow("frame", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break;
        

vs.release() 
result.release()
cv2.destroyAllWindows()